In [ ]:
!pip install gnss_lib_py
import numpy as np
import gnss_lib_py as glp
import matplotlib.pyplot as plt

In [ ]:
file_rinex = "/content/gnss_log_2024_11_21_12_57_35.24o"
file_nmea = "/content/gnss_log_2024_11_21_12_57_35.nmea"
file_txt = "/content/gnss_log_2024_11_21_12_57_35.txt"

In [ ]:
fix_data = glp.AndroidRawFixes(file_txt)

In [ ]:
from datetime import datetime
from pytz import timezone

# parameters to change #<----- change this!!!
textfile_date = datetime(2024, 11, 21, 12, 57, 35, tzinfo=timezone('America/Los_Angeles'))
offset_ms = 392.32 * 1000   # additional offset #400 worked (ish)
use_precise = True      # use precise epehmeris (only available 1 week after meas)

# load raw data
def process_data(offset_ms, use_precise=False):
    raw_data = glp.AndroidRawGnss(input_path=file_txt,
                                  filter_measurements=True,
                                  measurement_filters={"sv_time_uncertainty" : 500.},
                                  verbose=True)

    # first correct to the roughly correct time
    gps_millisec_logged = raw_data["gps_millis"][0]
    datetime_logged = glp.gps_millis_to_datetime(raw_data["gps_millis"])[0]
    true_gpsmillisec = glp.datetime_to_gps_millis(textfile_date)

    diff_gps_millisec = gps_millisec_logged - true_gpsmillisec
    raw_data["gps_millis"] = raw_data["gps_millis"] - diff_gps_millisec   # correct to the log file date

    print("datetime (logged):", datetime_logged)
    print("gps_millis (logged): {0:4.2e}".format(gps_millisec_logged))
    print("gps_millis (log file time): {0:4.2e}".format(true_gpsmillisec))

    # additional offset to the gps millisec
    offset_data = raw_data.copy()
    offset_data["gps_millis"] = raw_data["gps_millis"] + offset_ms

    # extract gps states
    if use_precise:
      derived_sv_states = glp.add_sv_states(offset_data, source="precise", verbose=False)
      print(derived_sv_states)
    else:
      derived_gps = offset_data.where("gnss_id", "gps")
      sv_state_rows = ['x_sv_m', 'y_sv_m', 'z_sv_m', 'vx_sv_mps', 'vy_sv_mps', 'vz_sv_mps', 'b_sv_m']
      derived_sv_states = glp.add_sv_states_rinex(derived_gps)

    # correct satellite clock bias
    full_states = derived_sv_states.copy()
    full_states["corr_pr_m"] = full_states["raw_pr_m"] \
                            + full_states['b_sv_m']

    # remove all data with raw_pr_m > 1e8
    full_states = full_states.where("raw_pr_m", 3e7, "leq")
    glp.plot_metric_by_constellation(full_states, "gps_millis", "raw_pr_m")


    return wls_estimate, full_states


In [ ]:
full_states = process_data(offset_ms, use_precise)[1]

In [ ]:
import pandas as pd
if isinstance(full_states, dict):
    df = pd.DataFrame(full_states)
else:
    df = full_states

# Save the DataFrame to a CSV file
csv_file_path = "full_states.csv"  # Define the output file name
df.to_csv(csv_file_path, index=False)

print(f"Data has been saved to {csv_file_path}")